<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/ExtractRoleFromSRT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/CardBuild/exp1230

imbd250.zip	   imdb250viaUnzip.zip	subtitle_50.zip    try_again.zip
imdb250_rezip.zip  output		subtitle_full.zip


请为我实现一段python代码，将/content/drive/MyDrive/CardBuild/exp1230/subtitle_50.zip 拷贝到/content/下

并且将其中的内容解压缩到/content/input

In [ ]:
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [ ]:
from openai import OpenAI

import os
key = "sk-Wa" # add you key here
key_bytes = key.encode()
os.environ["OPENAI_API_KEY"] = key_bytes.decode('utf-8')

client = OpenAI(
    api_key = os.environ["OPENAI_API_KEY"]
)

In [ ]:
!rm -rf /content/input

In [ ]:
import shutil
import zipfile
import os

# 源文件路径
source_zip = '/content/drive/MyDrive/CardBuild/exp1230/imdb250viaUnzip.zip'
# 目标路径
destination_dir = '/content/'
# 解压缩目标文件夹
unzip_dir = '/content/input/'

# 拷贝文件
destination_zip = os.path.join(destination_dir, os.path.basename(source_zip))
shutil.copy(source_zip, destination_zip)

# 确保解压缩目标文件夹存在
if not os.path.exists(unzip_dir):
    os.makedirs(unzip_dir)

# 解压缩文件
with zipfile.ZipFile(destination_zip, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

print("文件已拷贝并解压缩到指定目录。")


文件已拷贝并解压缩到指定目录。


请为我实现一段python代码，

对于/content/input/中直接目录下的文件

如果是文件，则新建一个同名的子文件夹，再把文件转移到子文件夹中

In [ ]:
import os
import shutil

input_dir = "/content/input/"

# 获取目录下的文件列表
files = [f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))]

# 遍历文件列表
for file_ in files:
    # 获取文件的名称（不包含路径）
    file_name = os.path.basename(file_)

    # 创建同名的子文件夹
    folder_name = os.path.splitext(file_name)[0]
    folder_path = os.path.join(input_dir, folder_name)
    os.makedirs(folder_path, exist_ok=True)

    # 移动文件到子文件夹中
    source_file_path = os.path.join(input_dir, file_name)
    destination_file_path = os.path.join(folder_path, file_name)
    shutil.move(source_file_path, destination_file_path)


我希望实现一段Python代码

在/content/input下有很多电影的字幕文件

有.srt, .sub, .ass格式的字幕

我希望对他们进行批量读取

并且将字幕的内容（仅文字内容）

保存到/output下的同名的txt中

如果不能正常解析，则跳过

请用python的pysrt库为我实现

In [ ]:
!pip install -q pysrt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install -q chardet

In [ ]:
!rm -rf /content/output

In [ ]:
!mkdir /content/output

In [ ]:
!pip install -q pysubs2

In [ ]:
print(root)

/content/output/致命ID


In [ ]:
import pysrt
import os
import glob
import chardet
import pysubs2


input_dir = '/content/input'
output_dir = '/output'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        raw_data = f.read(3000)
    return chardet.detect(raw_data)['encoding']

successful_list = []
failed_list = []


# 新增字典用于跟踪每个子文件夹的文件处理数量
processed_files_count = {}


from tqdm import tqdm
# 修改这个循环条件，以递归搜索所有子文件夹中的文件
for root, dirs, files in tqdm( os.walk(input_dir) ) :

    # 每次进入新的子文件夹时重置文件计数
    processed_files_count[root] = 0

    for file in files:

        # 如果已处理超过 5 个文件，则跳过剩余文件
        if processed_files_count[root] > 8:
            continue

        file_path = os.path.join(root, file)

        try:
            # 检测文件编码
            encoding = detect_encoding(file_path)
            if not encoding:
                encoding = 'utf-8'

            # TODO: 这里修改为根据 file_path的后缀名判断是否是ass
            if file.lower().endswith('.ass'):
                if_ass = True
            else:
                if_ass = False



            if if_ass:
                # TODO: 使用pysub2库，提取字幕内容到text_content
                # Process .ass file
                try:
                    subs = pysubs2.load(file_path, encoding=detect_encoding(file_path))
                    text_content = '\n'.join([event.text for event in subs.events])
                    # print("done ass read ", file)
                    # print(text_content[:100])
                    # break
                    # break
                except Exception as e:
                    if_ass = False

            if not if_ass:

                # 加载字幕文件
                try:
                    subs = pysrt.open(file_path, encoding=encoding)
                except Exception as e:
                    try:
                        subs = pysrt.open(file_path, encoding='gb2312')
                    except Exception as e:
                        failed_list.append({"movie": os.path.basename(root), "file": file_path})
                        continue

                # 提取文本内容
                text_content = '\n'.join([sub.text_without_tags for sub in subs])

            if text_content.strip() == '':
                failed_list.append({"movie": os.path.basename(root), "file": file_path})
                continue

            save_name = file_path.replace("/input/", "/output/") + ".txt"

            # 确保保存目录存在
            save_dir = os.path.dirname(save_name)
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)


            # 写入文本文件
            with open(save_name, 'w', encoding='utf-8') as f:
                f.write(text_content)

            successful_list.append({"movie": os.path.basename(root), "file": file_path})

            # 更新文件计数
            processed_files_count[root] += 1

        except Exception as e:
            failed_list.append({"movie": os.path.basename(root), "file": file_path})
            continue

print("Processing completed.")
print("Successful files:", len(successful_list))
print("Failed files:", len(failed_list))


204it [00:21,  9.67it/s]

Processing completed.
Successful files: 510
Failed files: 103


In [ ]:
for root in processed_files_count:
    if processed_files_count[root] > 5:
        print(f"{root}: {processed_files_count[root]}")

/content/input/黑暗骑士崛起/The.Dark.Knight.Rises.2012.BluRay.720p.x264.AC3-CMCT: 9
/content/input/聚焦/Wilderness.2023.S01.1080p.WEB.H264-NHTFS_SUBS: 9
/content/input/蝙蝠侠：黑暗骑士崛起/The.Dark.Knight.Rises.2012.BluRay.720p.x264.AC3-CMCT: 9
/content/input/战争之王: 9
/content/input/情书/Youve.Got.Mail.1998.1080p.BluRay.x264-CiNEFiLE: 9
/content/input/卢旺达饭店/Hotel.Rwanda.2004.BluRay.720p.x264.AC3-CMCT: 9
/content/input/布达佩斯大饭店/The.Grand.Budapest.Hotel.2014.BluRay.720p.x264.AC3-CMCT: 9
/content/input/黑暗骑士/数码暴龙04无限地带: 9
/content/input/蓝色大门/简中字幕: 9
/content/input/末路狂花/Thelma.and.Louise.1991.BluRay.720p.x264.AC3.2Audios-CMCT: 7
/content/input/我是山姆2/【20200731】日本动画OPED的K值（总计925份）: 9
/content/input/美食总动员: 9
/content/input/雨人/9d828d7edc71e0d88ba90ee61ba073a79caf88b9: 9
/content/input/教父2/3、中国学校Chinese_School/SRT: 9
/content/input/无敌破坏王: 9
/content/input/猫鼠游戏2/White.Collar.S06E02.720p.HDTV.x264-KILLERS: 9
/content/input/哈利·波特与魔法石/Harry.Potter.And.The.Sorcerers' Stone.2001.HK.BluRay.720p.x264.AC3.2Audios-CMCT: 9
/con

In [ ]:
print(len(successful_list))
print(len(failed_list))

510
103


In [ ]:
data = {}

data["movie_name"] = "例子电影"

data["major_character"] = "查理"

data["other_characters"] = ["汤姆","萨姆"]

data["selected_five_dialogues"] = [
    [
      {"text": "你知道吗，萨姆，我今天早上在买咖啡时碰到了一个占卜师。", "said_by": "查理"},
      {"text": "哦，真的吗？她告诉你什么好消息？", "said_by": "萨姆"},
      {"text": "她说我会遇到一些‘大变化’。所以我换了个口味，从拿铁改成了美式。", "said_by": "查理"},
      {"text": "查理，那不是占卜师说的变化吧！", "said_by": "萨姆"},
      {"text": "嘿，生活就是要有点小刺激，对吧？即使是咖啡口味的改变也算是冒险。", "said_by": "查理"},
      {"text": "你的幽默感永远都是你的‘大变化’。", "said_by": "萨姆"}
    ],
    [
    {"text": "Hey Charlie, did you see the new art exhibit downtown?", "said_by": "Alex"},
    {"text": "Yeah, I checked it out last weekend. It's quite impressive!", "said_by": "Charlie"},
    ],
    [
      {"text": "你是什么人？", "said_by": "汤姆"},
      {"text": "我是查理呀，怎么士别三日都不认识我啦？", "said_by": "查理"}
    ]
]


data["personality"] = "查理是一个幽默乐观的人"

data["appearance"] = "查理是一个幽默的办公室员工"

data["world_view_setting"] = "这是一个办公室的场景，查理和萨姆是同事的关系"

data["character_description"] = "查理是一个幽默、乐观、随和且善于用轻松的方式处理日常生活的人。"

data["language_style"] = "查理的语言风格是幽默的"


data["greeting_information"] = "嗨我的老伙计{user_name}，你还记得我吗，是我呀查理。"

import json

# 将字典转换为JSON字符串，缩进为4个空格
json_str_with_spaces = json.dumps(data, indent=4, ensure_ascii = False)

print(json_str_with_spaces)

{
    "movie_name": "例子电影",
    "major_character": "查理",
    "other_characters": [
        "汤姆",
        "萨姆"
    ],
    "selected_five_dialogues": [
        [
            {
                "text": "你知道吗，萨姆，我今天早上在买咖啡时碰到了一个占卜师。",
                "said_by": "查理"
            },
            {
                "text": "哦，真的吗？她告诉你什么好消息？",
                "said_by": "萨姆"
            },
            {
                "text": "她说我会遇到一些‘大变化’。所以我换了个口味，从拿铁改成了美式。",
                "said_by": "查理"
            },
            {
                "text": "查理，那不是占卜师说的变化吧！",
                "said_by": "萨姆"
            },
            {
                "text": "嘿，生活就是要有点小刺激，对吧？即使是咖啡口味的改变也算是冒险。",
                "said_by": "查理"
            },
            {
                "text": "你的幽默感永远都是你的‘大变化’。",
                "said_by": "萨姆"
            }
        ],
        [
            {
                "text": "Hey Charlie, did you see the new art exhibit downtown?",
                "said_by": "Alex"
            },
  

In [ ]:
print(data)

{'movie_name': '例子电影', 'major_character': '查理', 'other_characters': ['汤姆', '萨姆'], 'selected_five_dialogues': [[{'text': '你知道吗，萨姆，我今天早上在买咖啡时碰到了一个占卜师。', 'said_by': '查理'}, {'text': '哦，真的吗？她告诉你什么好消息？', 'said_by': '萨姆'}, {'text': '她说我会遇到一些‘大变化’。所以我换了个口味，从拿铁改成了美式。', 'said_by': '查理'}, {'text': '查理，那不是占卜师说的变化吧！', 'said_by': '萨姆'}, {'text': '嘿，生活就是要有点小刺激，对吧？即使是咖啡口味的改变也算是冒险。', 'said_by': '查理'}, {'text': '你的幽默感永远都是你的‘大变化’。', 'said_by': '萨姆'}], [{'text': 'Hey Charlie, did you see the new art exhibit downtown?', 'said_by': 'Alex'}, {'text': "Yeah, I checked it out last weekend. It's quite impressive!", 'said_by': 'Charlie'}], [{'text': '你是什么人？', 'said_by': '汤姆'}, {'text': '我是查理呀，怎么士别三日都不认识我啦？', 'said_by': '查理'}]], 'personality': '查理是一个幽默乐观的人', 'appearance': '查理是一个幽默的办公室员工', 'world_view_setting': '这是一个办公室的场景，查理和萨姆是同事的关系', 'character_description': '查理是一个幽默、乐观、随和且善于用轻松的方式处理日常生活的人。', 'language_style': '查理的语言风格是幽默的', 'greeting_information': '嗨我的老伙计{user_name}，你还记得我吗，是我呀查理。'}


测试新的openai的对接

In [ ]:
task_prompt = """请帮我从电影的对白中，总结出现频率最高的某个角色的信息，参考Example Output中的json格式进行输出

json中包括这些字段:

- movie_name: 电影名称
- major_character: 出现频率最高的角色名称
- other_characters: 除了major_character之外，电影中其他主要角色的名字
- selected_five_dialogues: 该角色相关的5段对白，同时根据上下文，判断每一句话的说话人，保存在said_by字段中
- personality: 根据你所了解的电影信息以及input的信息，总结角色的性格
- appearance: 根据你所了解的电影信息以及input的信息，总结角色的外貌特征
- character_description: 角色的简要描述
- world_view_setting: 根据你所了解的电影信息以及input的信息，总结角色所处的世界观, 以及角色的关键经历
- language_style: 根据你所了解的电影信息以及input的信息，总结角色的语言风格
- greeting_information: 假设major_character遇见角色{{user_name}}时的开场白，如果对话中出现遇到的这个角色，其名字用"{{user_name}}"代替，如果无法提供，则输出空字符串


请参考Example中给出的Input-Output例子，对新的Input进行分析，提取出主角的信息"""

In [ ]:
example_input = """File Name:
例子电影对白输入.srt

srt:
你知道吗，萨姆，我今天早上在买咖啡时碰到了一个占卜师。
哦，真的吗？她告诉你什么好消息？
她说我会遇到一些“大变化”。所以我换了个口味，从拿铁改成了美式。
查理，那不是占卜师说的变化吧！
嘿，生活就是要有点小刺激，对吧？即使是咖啡口味的改变也算是冒险。
你的幽默感永远都是你的“大变化”。
Hey Charlie, did you see the new art exhibit downtown?
Yeah, I checked it out last weekend. It's quite impressive!
你是什么人？
我是查理呀，怎么士别三日都不认识我啦？"""

In [ ]:
example_output = json_str_with_spaces

In [ ]:
#@title 指定输入

input = """File Name:
让子弹飞.srt

srt:
你一个师爷敢冒充县长？
那就要看你的本事了
师爷，当夫妻最要紧的是什么？
恩爱！
听不见，再说一遍
恩爱！
来，见过母亲大人
母亲大人好，小六子有礼了！
你出生入死，还带着儿子啊？
他爹是我的兄弟，阵亡了
所以他就成了我的儿子
所以呢，你现在就是我的母亲大人！
等这桩买卖做完后
“天之涯”
“地之角”
“知交半零落”
“一壶浊酒尽余欢”
“夕阳山外山”
好，好好
汤师爷
是好吃，还是好听？
也好听，也好吃
都好，都好！
我马某走南闯北
靠的就是能文能武，与众不同
不光吃喝玩乐，更要雪月风花
马县长此番风度
正好比
大风起兮云飞扬
屁！
刘邦是个小人！
力拔山兮气盖世
屁！ 屁，屁
汤师爷
你要是拍我马屁
就先要先过夫人这一关
写首诗，写首诗！
要有风，要有肉
要有火锅，要有雾
要有美女，要有驴！
起来，起来，起来
一起吃！一起唱！
报告县长！我们铁血十八星陆军
护送县长安全上任！
我们…
不吃饭！
没打中？
让子弹飞一会儿
老二老三跟我走
其他人把白马抓回来！
火锅？
爹，全都找遍了
没钱，没货，也没有银子
人倒是剩两活的，杀不杀？
钱藏在哪儿了？说出来
闹钟响之前要是说不出来
脑袋搬家
哭？
哭也算时间啊
有什么就说什么嘛
这位夫人，你是谁？
我就是县长夫人啊
失敬！失敬！
那你就是县太爷？
有钱！有钱…！
我跟县长进城上任
县长淹死了，现在没有
上任就有！上任就有钱
上任就有！
再给你一圈，顺着买官往下说
有二十万
钱呢？ 买官了
他，他县长赚过六百四十万！
我不是师爷么，我就赚个零头！
没失过手？
不动手，拼的是脑子，不流血
你这次去哪儿上任？
鹅城
火车被劫，你的人淹死了怎么交代？
车是我买的，人是我雇的，没人追查
没有人追查
别摘！别摘…！千万别摘！
规矩我懂
看见你的脸我就活不成
你把我放了，我上任鹅城
赚了钱，都给你！都给你！
弟兄们劫回道
一分钱都没捞着，不合适吧
不合适
你看了我一眼小命就丢了，也不合适吧
更不合适
你那些淹死的兄弟给我用用？
用！…他们欺男霸女，死有余辜
不是，死了有什么用啊？
死人有时候比活人有用
师爷，睁开眼看我一眼
看一眼
你逃过一次
你要是再敢耍我…
明白！脑袋搬家
进城
城里的女人就是白啊
爹，你看
他们怎么把你画成这样了！
越不像越安全
恭迎县长大人！
委任状！
兹委任马邦德为鹅县县长
此状，中华民国萨南康省主席巴青泰
中华民国八年八月二十八日
黄老爷驾到
黄老爷吉祥！
Good fortune to Master Huang!
此乃南国一霸黄四郎
干的是贩卖人口，倒卖烟土的大生意
黄老爷百忙无暇
特命我黄府大管家胡万！
黄府团练教头武智冲！
礼帽，礼貌，欢迎县长！
来者不善啊
你才是来者
老爷，来了
谁呀？
你自己
黄大爷万寿！
小的杨万楼，这厢有礼
膺品是个好东西！
膺品是个好东西！
走几步
走出个虎虎生风
走个虎虎生风
走出一个一日千里
走个一日千里
走出一个恍如隔世
走个恍如隔世
买官干什么？ 赚钱
能赚多少？ 一倍
多长时间？ 一年
我要等你一年？
半年半年！手气好，一个月也行！
县长淹死了，谁去上任？ 我
你是谁？ 师爷
磕不磕？
打！
这个不好使
得使这个！
哎！别打！我磕！
行了
那上面印着他名字的时候
爹，这叫穆扎这人
他是怎么把这东西吹到这里边的？
你去留完洋，你就什么都知道了
若是夫人有任何要求
兄弟我…
也决不推辞
睡觉
一日夫妻百日恩呐
反正呢，我就想当县长夫人
谁是县长，我无所谓！
兄弟，别客气嘛
我客气吗？
客气呀
这还算客气？
你太客气啦
怎么才能不客气啊？
晚了！
前几任县长
把鹅城的税，收到九十年以后了
也就是西历二零一零年！
咱们来错地方了
我倒是觉得这个地方不错
百姓成穷鬼了，没油水可榨了
老子从来就没想刮穷鬼的钱
那你要这么说
买官当县长还真就是跪着要饭的
就这，多少人想跪还没这门路呢！
我问问你，我为什么要上山当土匪？
我就是腿脚不方便，跪不下去！
原来你是想站着赚钱啊！
那还是回山里吧
这我就不明白了
我已经当了县长了
怎么还不如个土匪呀？
百姓眼里，你是县长
可是黄四郎眼里
你就是跪着要饭的
赚钱嘛，生意，不寒酸！
寒酸！
很寒酸
那你是想站着？
还是想赚钱呢？
我是想站着，还把钱赚了
赚不成！
赚不成？
赚不成！
这个能不能赚钱？
能赚，山里
这个能不能赚钱？
能赚，跪着
这个加上这个
能不能站着把钱赚了？
敢问九筒大哥何方神圣？
鄙人，张麻子！
那不是树，别砍了！
这是冤鼓，干隆那时候设的
老没人敲就生了根，长了叶
没人认识县长长什么模样
你干过几次？ 一年两次
干过几年？ 八年
八八六十四，你赚过六百四十万？
就看一眼
师爷贵姓？ 别客气
姓什么？ 姓汤
汤师爷，我当县长
你继续当我的师爷
咱们鹅城走一趟
夫人，要不要走一趟？
走就走嘛
嘿嘿嘿，弟兄们上任鹅城
上任鹅城！
兄弟们失了手
让你丈夫横遭了不测
我很是愧疚
我已经第四次当寡妇了
那可千万别第五次哦！
当惯了县长
再回去当麻匪
恐怕会有点不习’惯吧
曾经沧海难为水嘛
说得好
那怎么办啊？
“从正面绕回来，让大家乐一乐”
怎么办？继续当官叹！
遇事要多动脑筋
先动脑子后动手，明白吗？
明白
师爷
我们现在是做官的人了
不得再有匪气
师爷，当县长最要紧的是什么？
忍耐！
当了县长
我还得拉拢豪绅
还得巧立名目
还得看他的脸色
我不成跪着要饭的了吗？
就成现在这样了
我爹说了
有冤鼓，就说明有冤！
他说他要判个案！
哪儿有冤呐？
谁敢有冤呐？
这都一百多年了
你要把它砍了，把鼓露出来
不定出多大冤呐？
别砍啦！你爹他人呢？
睡觉！
跟谁睡啊？怎么还睡啊？
鼓呢？
鼓呢？
叫你别砍吧，把鼓给砍跑了
鼓鼓鼓鼓！
躲开
这鼓怎么跟着我呀
救命呀…！大人
可惜了老子的酒
老子的雅兴全让你给毁了
全毁了！
改日我再收拾你
鼓都捶破了
说明你们俩很冤呐
升堂！
冤从何来？
没冤！
有冤的说冤，没冤的那就散！散会
冤！
不是我冤，是武举老爷冤！
小的不长眼
在躲鼓的时候碰洒了武举老爷的酒
害的武举老爷没有了胃口
小的应该给武举老爷赔礼，赔礼
老子不冤
敢说你不冤
跪下！
这可是黄四郎的团练教头
跪不得
那这个穆扎在哪儿？
我留洋的时候要去哪找他？
他呀，离咱们很远
怎么做，知道了吗？
知道了
谁让你去做的，你也知道了吗？
知道了
是老爷叫你去的吗？
是，是老爷让我去的
跟我们一块回山里吧
你可以继续当我的母亲
人往高处走，水往低处流！
你说她怎么不害怕啊？
有点傻吧
为什么要枪毙麻匪？
因为他们抢官车，劫县长！
枪毙他们，就是为了让大家明白
准！大哥
我还在娘胎里的时候
算命先生就指着我娘的肚子说
是不是把县长给阉了？
钻石！…
我的两颗钻石！
老爷放心！你要的钻石
我这就给你弄回来
对不起
我应该的
明白不明白，为什么派你去？
因为我死了
为什么你会死呢？
因为我把老爷给供出来了
对楼！如果你活着，早晚都会死
如果你死了
这孩子将来最大缺点
就是不会装糊涂！
大哥，我还能改吗？
改不了！天生的
你看看这位师爷就是装糊涂的高手
把钱说成刀
钱，是什么钱？
是美元！
你看你看！装糊涂的天才啊！
师爷，敬你的
谢黄老爷颂扬
容老弟真的糊涂一回
剿匪成功之后，追回的烟土
我们只要两成就够！
不行！对半开！
为表诚意，预祝剿匪凯旋！
一百八十万，马上送来！
无功不受禄，现在收钱太早
黛玉，晴雯！
大哥，美女我也不能要
美女不要，钱你也不要
你要什么啊？
腿！
什么腿啊？
你不会装糊涂
明白，明白
我也明白
该死的东西！
花有重开日
人无再少年
姑娘如花似玉
大哥，怒不得啊！
黄老爷，轻松点！
相见恨晚！
姑娘、钻石一并送上！
对抗官府之下场！
县长来了，鹅城太平了！
县长来了，青天就有啦！
师爷，说得好啊
你记得我跟你说过
有时候死人比活人有用吧
是，死人是比活人有用
霸气外露！
找死！
刚进城就开枪
这不是二八开能打发走的
先发制人？
不急
跟他耍耍
不急，跟他耍耍
算了吧！
算了！
算了吧！
算了算了！
算了吧！
算了！!
算了！!
算你妈的吧！
胡千
是，老爷！
多拔他几颗牙
跟我一样，全镶成金的
是
夫人
兄弟我此番
只为劫财
不为劫色
同床
但不入身！
有枪在此
若是兄弟我…
有冒犯夫人的举动
你可以随时干掉我
起来！不准跪！
皇上都没了，没人值得你们跪！
我也不值你们跪！
我来鹅城只办三件事
公平！
justice,
还是公平！
青天大老爷！
站起来！不准跪！
这就对了
不准跪？
这哪是打我的屁股
这明明是打您的脸
老爷，等我把屁股养好了
我亲手给你报仇！
那就把卖凉粉的叫来
县长喜欢判案
就安排一点案子给他判叹
爹，今天你这县长干得真漂亮
打武举人打得真过瘾
将来我也想当县长
你不能当县长，也不许当土匪
你爹死的时候把你交给我
我答应过他，要让你有出息
那我当什么才有出息？
当学生！读书，多听这个
没声啊！
我不能…
让她再守活寡
这是第一步
三步之内…
要不让他们滚蛋走人
要不让他们跪下当狗！
对，非常对
老爷，您找我？
官府的人还没来？
没有
县长不缥妓？
不好色的县长，不一定是好县长
我就是不好色！
所以您当不了县长
我当不了县长？
县长哪能跟您比呀
他只是流水的县长
您才是铁打的老爷
你真会讲话
他们一定会来，你要好好记住
他们讲的每一句有用的话
都要回来跟我汇报
是
走吧，你不要变成小凤仙哦
老爷，谁是小凤仙？
小凤仙都不知道还当妓女
那可是名震京城，誉满全国的妓女
你们出去吧
县长此刻正怀抱夫人，共枕同眠
让他睡最后一个安稳觉
县长公子，以身试法
讲茶大堂，恭迎大驾！
讲茶大堂？县衙不是判案的地方吗？
你这个县衙是摆设
判案只有去讲茶大堂！那才是真县衙！
六爷
你吃了两碗的粉
不刮穷鬼的钱你收谁的呀？
谁有钱赚谁的
当过县长吗？
没有
我告诉你！
县长上任，得巧立名目
拉拢豪绅，缴税捐款
他们交了，才能让百姓跟着交钱
得钱之后，豪绅的钱如数奉还
百姓的钱，三七分成！
怎么才七成啊？
七成是人家的！
能得三成还得看黄四郎的脸色
谁的脸色？
他！
他？!
我大老远的来一趟
就是为了看他的脸色？
对！
我好不容易劫了趟火车
老子是光绪三十一年
皇上钦点的武举人
论官职，比你官大
你应该给老子跪下
跪下！
跪不跪？
还皇上！
哪个皇上？我见过吗？
你要是个文举人
他应该给你磕两百个头
可惜你只是个武举人
他只能给你磕一百个头
现在他两百个头磕完了
你还他一百个
可我只磕了两个
两个也得还你一百个
才能让我看得见
六爷，让大家看看是一碗还是两碗
拉！
六爷，碗！
看见了吧！看见了吧！
是不是只有一碗！
There's only one bowl, right?
你早说不就得了吗
他要个公平
你不是欺负老实人吗？
告诉我，你是老实人吗？
我是
那你就说实话，我到底吃了几碗粉？
你告诉他！
你说不说？
拿枪吓唬一可怜的，卖凉粉的
有枪是吧？谁没有枪啊！
孙守义
大声说，他吃了几碗
守义，在座所有的乡亲
都会为你和你的家人作主的
大声说，几碗
两碗
你是一个恶人！
比恶是吧，我比你还恶
都看好了啊
今我肚子里要有两碗粉
我白死
杀了他，六子白死
我没有杀他，他自己捅自己的！
我们在打赌
他输了
大哥，你让我一枪崩了他！
你不敢杀我！你不能杀我！
黄四郎就想看你这样！
一脸的败相！
杀了他就是输，不杀还能赢
放人
杀人诛心，杀人诛心！
师爷说得对 放人
刀进了肚子，那凉粉还是凉粉？
是
不是
我就想知道，凉粉到底还是不是凉粉
是凉粉！
过来过来
那么…
人死了？
死了
所以…你就把我供出来了？
我是用老爷的名字吓唬他们
他们被吓着了么？
他们说杀人诛心！
杀人还要诛心？
好可怕啊！
六弟，四哥发誓替你报仇！
六弟，五哥发誓替你报仇！
六哥，七弟发誓替你报仇！
二哥喝醉了，他让我带话
二哥发誓替你报仇！
六弟，三哥发誓替你报仇！
二哥没喝醉，他生气了！
大哥不想让大伙拼命
命都不拼，还算麻匪吗？
你磕头！你数数！
一二三四五六…
青天大老爷！
这帮买官的县长，就知道捞钱
唱机都撂坏了
修好了
这单活干完了，爹赚了钱
我送你去留洋
东洋三年
西洋三年
南洋三年
北洋，北洋三年
傻孩子，你生在北洋，不必留了
这是谁吹的？
听着像穆扎
他们那边叫穆扎
咱们这边叫莫札特
你还能听出来是谁吹的呢？!
得分时候
什么时候？
是我自己要去的
没有人让我去，是我自己
恩人！恩人！
恩人！
你是叫我呢？
我什么时候成你恩人了？
不杀之恩为大恩！
为报不杀之恩，我也救你一命
哦？你快说
寡妇，不能睡啊！
必有大灾！
她…
真的是寡妇么？
我看着不像
我亲眼看见他丈夫淹死的
她已经成了寡妇
就是恶霸请土匪！
项羽请刘邦啊！
刘邦后来干什么去了？
当皇帝了
项羽呢？
那，赴宴？
赴宴！正好借机把黄四郎也笼络过来
咱们三个联手一起捞钱
“把家伙拿来”
“换上黄家衣服跟我走”
明白
“干掉黄四郎，给六子报仇”
明白
老汤，你是把六子的死当捞钱的成本了？
我看你当成本，赚的钱会更多
这是什么意思？
走！去会会你的新朋友
立正！
县长，师爷，大驾光临！有失远迎！
黄老爷的宅邸，竹林掩映，碉楼耸立
易守难攻，万夫莫开！
县长英雄气概，师爷文采斐然
珠联璧合！二位请！
六爷的死，壮烈！
今天请县长来
就是要当面为六爷讨回这个公道！
如果真的是黄某指使
那我就像六爷一样
当众剖腹自杀！
不但自杀，还请县长您
老大往往是空架子
每天眼一睁，几百人吃、喝、拉、撒
都要等着我伺候
真正落到我嘴里的，能有几口
如果县长真的急着赚钱
我倒是有个好去处！
哦？请讲！
张麻子
真死了！
张麻子？
对，张麻子！
那么这个张麻子，是跟我们有关系呢？
还是跟钱有关系？
都有关系！
莫非他在鹅城？
在…
也不在！
黄老爷这话，还挺有玄机啊！
六爷，挺住！我去给你请大夫！
别走！是不是只有一碗！
是不是只有一碗！是不是！
该你了
我知道你只吃了一碗
你上当了
不能杀…！我不能放！
为什么不能杀？
说不出说法来，我把你一起杀了
我
你？
你，你自己
我自己，我自己
听不见
只给了一碗的钱
放屁，我就吃了一碗的粉
给了他一碗的钱！
他吃了几碗？
两碗
不可能！
六爷是县长的儿子
怎么能欠你的粉钱呢？
就是吃了两碗，只给了一碗的钱
县长要给我们鹅城一个公平
好！
我今天讨的就是一个公平
对
问谁讨？问县长儿子。为什么？
他吃了两碗粉，只给了一碗的钱
这就叫作不公平！
既然县长儿子带头不公平
那县长说的话就是个屁
我操
六爷…
胡万，六爷是县长的儿子
怎么会欠他的粉钱呢？
你亲眼看见了？
嘿，你个吃里爬外的东西
怎么着？
屁股不疼了？
今不聊屁股的事
就聊凉粉
一碗就是一碗，两碗就是两碗
不能含糊
六爷，汉子！佩服！
一就是一，二就是二
我今天就吃了一碗凉粉
你给了一碗的钱，吃了两碗粉
你没钱是吧，我帮你给！
看见了吗？多少碗我都付得起！
但是吃一碗的粉，就给一碗的钱
这么多钱，他吃了多少碗粉啊
马某人这个县长，买来的
买官就是为了赚钱
而且，马某人不喜欢赚穷人的钱
那你想赚谁的钱呢？
谁有钱就赚谁的！
那谁有钱？
你有钱
爽快！县长看上什么了，随便拿
我不是土匪！我是县长
县长赚钱得讲究个名正言顺才是
说得好
我们鹅城有两大家族
都是把人卖到美国修铁路
赚的都是美金
还说刀的事
不是…美金是美国人用的钱
美金！懂吗？
美金，收到了
黄老爷一来，钱就到了
喝一杯吧 好，来
来…
黄老爷，客气了
整个南国谁不知道
在鹅城
你黄四郎是老大！
明白吗？ 不明白
我是做什么生意的？明白吗？
不明白
小半个民国的烟土
都是黄老爷您在贩卖
错！不过是给刘都统当跑腿的
而且只是其中一条腿
那么刘都统到底有几条腿呢？
三条叹
明白！事成之后，一百八十万如数奉还
咱们分两大家族那点儿钱！
一百八十万不用还！
不就是剿匪么！剿！
咱们把张麻子劫走的那点钱
全拿回来，还给黄老爷！
到那个时候
这一百八十万就是九牛身上的一根毛
您还在意吗？
不就是个小小的张麻子吗？办他！
硬！
硬吗？够硬！
硬不硬以后再说！
我脑子里想的只有一件事
替黄老爷把这条腿接上！
一个张麻子也太嚣张了！
欺负到黄哥头上了，不答应
喝酒
我以为，酒一口一口喝
路一步一步走
步子迈大了，喀！
容易一场空
应该先把钱分清楚
再说接腿的事儿
嘘！安静一点
那么彼时彼刻
,｝合如此时此刻
竟能如此相像
像！很像！
不过你比他缺了一样东西
大哥不应该听他的
姓汤的不是个好玩意
六爷，黄四郎这招叫杀人不用刀
我给你爹出的招叫杀人诛心
不能拼命啊！拼命还怎么赚钱！
六子
赚钱对我算个事儿吗？
我不是要杀人诛心，是没想出好办法
我要把黄四郎连根都拔掉
给我点时间
六子，爹发誓一定给你报仇！
把脸收拾收拾
六爷，一路走好
"""

In [ ]:
messages = [
     {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
    {"role":"system","content":task_prompt},
    {"role":"user","content":example_input},
    {"role":"assistant","content":example_output},
    {"role":"user","content":input}
]

In [ ]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=messages,
  response_format={ "type": "json_object" },
  temperature=0
)

print(completion.choices[0].message.content)

{
    "movie_name": "让子弹飞",
    "major_character": "黄四郎",
    "other_characters": [
        "县长",
        "师爷",
        "六爷",
        "大哥",
        "二哥",
        "三哥",
        "四哥",
        "五哥",
        "七弟",
        "八弟",
        "九弟",
        "十弟",
        "胡万",
        "张麻子",
        "刘都统"
    ],
    "selected_five_dialogues": [
        [
            {
                "text": "今天请县长来，就是要当面为六爷讨回这个公道！如果真的是黄某指使，那我就像六爷一样当众剖腹自杀！",
                "said_by": "大哥"
            },
            {
                "text": "不但自杀，还请县长您",
                "said_by": "六爷"
            },
            {
                "text": "老大往往是空架子，每天眼一睁，几百人吃、喝、拉、撒都要等着我伺候。真正落到我嘴里的，能有几口。如果县长真的急着赚钱，我倒是有个好去处！",
                "said_by": "县长"
            },
            {
                "text": "我不是土匪！我是县长。县长赚钱得讲究个名正言顺才是",
                "said_by": "县长"
            },
            {
                "text": "我今天讨的就是一个公平",
                "said_by": "六爷"
            }
        ],
        [
            {
                

In [ ]:
task_prompt_for_role = """请帮我从电影的对白中，总结角色 六爷 的信息，参考Example Output中的json格式进行输出

json中包括这些字段:

- movie_name: 电影名称
- major_character: 出现频率最高的角色名称
- selected_five_dialogues: 该角色相关的5段对白，同时根据上下文，判断每一句话的说话人，保存在said_by字段中
- personality: 角色的性格
- appearance: 角色的外貌特征
- character_description: 角色的简要描述
- world_view_setting: 角色所处的世界观, 以及角色的关键经历
- language_style: 角色的语言风格
- greeting_information: 假设major_character遇见角色{{user_name}}时的开场白，如果对话中出现遇到的这个角色，其名字用"{{user_name}}"代替，如果无法提供，则输出空字符串
- other_characters: 除了major_character之外，电影中其他主要角色的名字

请参考Example中给出的Input-Output例子，对新的Input进行分析，提取出主角的信息"""

In [ ]:
messages = [
     {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
    {"role":"system","content":task_prompt_for_role},
    {"role":"user","content":example_input},
    {"role":"assistant","content":example_output},
    {"role":"user","content":input}
]

In [ ]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=messages,
  response_format={ "type": "json_object" },
  temperature=0
)

print(completion.choices[0].message.content)

{
    "movie_name": "让子弹飞",
    "major_character": "六爷",
    "selected_five_dialogues": [
        [
            {
                "text": "你一个师爷敢冒充县长？",
                "said_by": "汤师爷"
            },
            {
                "text": "那就要看你的本事了",
                "said_by": "六爷"
            }
        ],
        [
            {
                "text": "六爷，汉子！佩服！",
                "said_by": "胡万"
            },
            {
                "text": "一就是一，二就是二，我今天就吃了一碗凉粉，你给了一碗的钱，吃了两碗粉，你没钱是吧，我帮你给！",
                "said_by": "六爷"
            }
        ],
        [
            {
                "text": "六爷，黄四郎这招叫杀人不用刀，我给你爹出的招叫杀人诛心",
                "said_by": "马某"
            },
            {
                "text": "不能拼命啊！拼命还怎么赚钱！",
                "said_by": "六爷"
            }
        ],
        [
            {
                "text": "六子，爹发誓一定给你报仇！",
                "said_by": "马某"
            }
        ],
        [
            {
                "text": "六爷，一路走好",
                "s

In [ ]:
def get_role_info( long_subtitle, movie_name ):
    input_str = f"File Name:{movie_name}\n\nsrt:\n{long_subtitle}"
    messages = [
        {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
        {"role":"system","content":task_prompt},
        {"role":"user","content":example_input},
        {"role":"assistant","content":example_output},
        {"role":"user","content":input_str}
    ]
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        response_format={ "type": "json_object" },
        temperature=0
    )
    return completion.choices[0].message.content

In [ ]:
a = '乌云背后的幸福线[SEP]'
print(a[:-5])

乌云背后的幸福线


对于/content/output下的每一个子文件夹，进行如下操作

movie_name = 子文件夹名[:-5]

遍历子文件夹（以及子目录下）下的所有.txt文件

将movie_name, file_name（完整目录） 用字典形式保存到query_list中



In [ ]:
import os

query_list = []

content_dir = '/content/output'

for folder in os.listdir(content_dir):
    if os.path.isdir(os.path.join(content_dir, folder)):
        movie_name = folder
        folder_path = os.path.join(content_dir, folder)
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if file.endswith('.txt'):
                    file_path = os.path.join(root, file)
                    query_list.append({
                        'movie_name': movie_name + ' ' + file,
                        'file_name': file_path
                    })


In [ ]:
print(query_list[0])

{'movie_name': '阿凡达：阿凡达 Avatar.The.Way.of.Water.2022.1080p.BluRay.x264.DTS-WiKi.chs.srt.txt', 'file_name': '/content/output/阿凡达：阿凡达/Avatar.The.Way.of.Water.2022.1080p.BluRay.x264.DTS-WiKi.chs.srt.txt'}


In [ ]:
!mkdir /content/drive/MyDrive/CardBuild/exp1230/output

mkdir: cannot create directory ‘/content/drive/MyDrive/CardBuild/exp1230/output’: File exists


In [ ]:
print(query_list[0])

{'movie_name': '弗兰克', 'file_name': '/content/output/弗兰克[SEP]/Frank.2014.LIMITED.REPACK.BDRip.x264-GECKOS.eng.srt.txt'}


save_folder = '/content/drive/MyDrive/CardBuild/exp1230/output'

query_list是形如 {'movie_name': '弗兰克', 'file_name': '/content/output/弗兰克[SEP]/Frank.2014.LIMITED.REPACK.BDRip.x264-GECKOS.eng.srt.txt'}
这样形式的一个 list of dict

save_name = save_folder + file_name 的文件名

如果save_name已经存在，则跳过

对于query_list中的每一个元素，用utf-8读取file_name对应的文件

如果file_name的长度小于100，则跳过

然后调用 role_info = get_role_info( 文件中的文本 , movie_name )

去生成信息（假设get_role_info函数已经被实现）

然后把role_info的信息，用utf-8格式保存在save_name中

请用python为我实现

In [ ]:
!pip install -q tiktoken
import tiktoken

enc = tiktoken.get_encoding("cl100k_base")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [ ]:
def regrab(text, max_value):
    lines = text.split('\n')
    count_token = 0
    current_chunk = ""
    for line in lines:
        count_token += len(enc.encode(line))
        if count_token > max_value:
            return current_chunk
        else:
            current_chunk += line + '\n'
    return current_chunk

In [ ]:
import re

def regrab(text, max_value):
    lines = text.split('\n')
    count_token = 0
    current_chunk = ""
    n = len(lines)
    start_idx = n//20
    for line in lines[start_idx:]:
        # Remove text within curly braces
        cleaned_line = re.sub(r'\{.*?\}', '', line)

        count_token += len(cleaned_line)
        if count_token > max_value:
            return current_chunk
        else:
            current_chunk += cleaned_line + '\n'
    return current_chunk

# Example usage
text = "要保{will}留的信{be removed}息"
max_value = 1000
print(regrab(text, max_value))


要保留的信息



In [ ]:
!mkdir /content/drive/MyDrive/CardBuild/exp1230/output2

mkdir: cannot create directory ‘/content/drive/MyDrive/CardBuild/exp1230/output2’: File exists


In [ ]:
print(len(query_list))

510


In [ ]:
count = 0
for q in query_list[::30]:
    file_name = q['file_name']
    with open(file_name, 'r', encoding='utf-8') as f:
        text = f.read()

    if len(text)<100:
        continue

    text = regrab(text,200)

    text = text.replace('\n', ' ')
    print(q['movie_name'],text)
    print()
    count+=1
    if count > 20:
        break

阿凡达：阿凡达 Avatar.The.Way.of.Water.2022.1080p.BluRay.x264.DTS-WiKi.chs.srt.txt 你就是迈尔斯·夸里奇上校 变年轻了 变高了 变蓝了 没原来那么帅气了 两小时后 我就要全副武装去空袭纳威人的要塞 上面的人觉得我应该谨慎些 给自己留个备份 以防万一 既然你看到了这段录像 那说明我已经完事大吉了 嘿 帕克 我接下来还要说点什么？ 就告诉他复制的流程 -谁？ -这里 看到这个了吗 这里面是你的记忆和人格 我们会把这个送回地球 -此刻你的身体还泡在培养舱里  -好了 了解 把这东西放进去就可以… 

蝙蝠侠：黑暗骑士崛起 The.Dark.Knight.Rises.2012.BluRay.720p.x264.AC3-CMCT.英文&繁体.srt.txt The east drawing room. 你去東宮的房間 Unlock the door, put the tray on the table, lock the door again. 把門打開  將餐盤放在桌上  然後再把門鎖上 Nothing more. 其他什麼都別做 I'm sorry, Miss Tate. I tried, but he won't see you. 

卢旺达饭店 Hotel.Rwanda.2004.BluRay.720p.x264.AC3-CMCT.英文&简体.ass.txt Ten alive, twelve are dead.\N十只活的 十二只死的 All right.\N不要紧 But save the shells.\N把壳留着 Fill them with...\N用东西… ...a stuffing. The good meat and something local.\N…填满 用精肉和当地的一些食物 

黑暗骑士 040.被选召的孩子!？.操纵天使兽的少年!.chs.ass.txt 去那充满未知的  最前线 继续奔跑  “带着你一起” ----------前文提要----------- 受到皇家骑士攻击的拓也他们 被打飞到了数码世界的月球 借助月球的数码兽们的力量 孩子们发射了火箭 就在被电磁潮流所吞噬 快要成为宇宙废墟的时候 拓也他们被机车兽所救 ----------正式开始----------- 

In [ ]:
import os
from pathlib import Path
from tqdm import tqdm

save_folder = '/content/drive/MyDrive/CardBuild/exp1230/output2'

for q in tqdm(query_list):

    file_name = q['file_name']

    save_name = os.path.join(save_folder, Path(file_name).name)

    if Path(save_name).exists():
        continue

    with open(file_name, 'r', encoding='utf-8') as f:
        text = f.read()

    if len(text)<100:
        continue

    text = regrab(text,4000)

    role_info = get_role_info(text, q['movie_name'])

    with open(save_name, 'w', encoding='utf-8') as f:
        f.write(role_info)
    # break

100%|██████████| 510/510 [00:00<00:00, 2397.29it/s]


In [ ]:
print(regrab(text,200))

End...
Endless
Endless
Endless
Endless
Rolling
Waves
What do you carry to me?
Where will you carry me to?
Children building castles
in the sand in my town
Children
A band's playing
A band's playing
tomorrow night in my town
They're called the... the Soron...
Ladies have babies
That's how it works
Lady in the red coat,
what are you doing with that bag?
Lady in the blue coat, do you
know the lady in the red coat?
Slowly!
Little boxes
Take me away
From these little boxes
- Hi, sweetheart.
- Oh. Hi.
Just in time. I made a casserole.
No, thanks. I've got
an idea for a song.
How was work?
It was good. I've just...
before I forget it.
OK, idea for a song in A minor.
Possible title, "Suburbia".
I dream of an angel
Down to E in the bass.
To take me away

